In [1]:
# 19/07)2020
# Francisco Dominguez

In [29]:
import random
import os
import json
import pickle
import numpy as np
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
#from nltk.stem import SnowballStemmer
#stemmer = SnowballStemmer('spanish')

In [30]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
keras.__version__

'2.2.4-tf'

In [101]:
class NLPModel(object):
    def __init__(self):
        self.chatBot=None
    def setChatBot(self,cb):
        self.chatBot=cb
class NLPANN(object):
    pass
class NLPANNkeras(NLPANN):
    def __init__(self,nlpModel):
        self.nlpModel=nlpModel
        self.ann=None
    def train(self,train_x,train_y):
        self.ann = Sequential()
        self.ann.add(Dense(25, input_dim=train_x.shape[1]))     # densidad de la primera capa de neurona y tipo de entrada
        self.ann.add(Dropout(0.5))                                   # convierte a 0 la mitad de 1 en el entrenamiento
        self.ann.add(Dense(25))                                      # densidad de la primera capa de neurona
        self.ann.add(Dropout(0.5))                                   # convierte a 0 la mitad de 1 en el entrenamiento
        self.ann.add(Dense(train_y.shape[1], activation='softmax'))  # densidad de la salida
        self.ann.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
        #self.ann.build()
        self.ann.summary()
        self.ann.fit(train_x, train_y, epochs=500, batch_size=8)   # entrena el modelo
    def save(self,filename):
        self.ann.save(filename)
    def load(self,filename):
        self.ann = load_model(filename)
    def predict(self,sentenceBow):
        p=self.ann.predict(np.reshape(sentenceBow,(1,-1)))
        return p
class NLPModelBoW(NLPModel):
    def __init__(self):
        # Bag of Words data
        self.words=[]
        self.classes=[]
        self.documents=[]
        self.ignore_words=['?']
        # MLP data (this could be in a different object)
        self.ann=NLPANNkeras(self)
        self.train_x = []
        self.train_y = []
    # return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
    def clean_up_sentence(self,sentence):
        # tokenize the pattern
        sentence_words = nltk.word_tokenize(sentence)
        # stem each word
        sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
        return sentence_words
    def bow(self,sentence, show_details=False):
        # tokenize the pattern
        sentence_words = self.clean_up_sentence(sentence)
        # bag of words
        bag = []
        # create our bag of words array
        for w in self.words:
            bag.append(1) if w in sentence_words else bag.append(0)
        return(np.array(bag))
    def buildData(self):
        self.buildBowData()
        self.buildTrainingData()
    def buildBowData(self):
        self.words=[]
        self.classes=[]
        self.documents=[]
        for intent in self.chatBot.intents:
            for pattern in intent.patterns:
                # tokenize each word in the sentence
                w = nltk.word_tokenize(pattern)
                # add to our words list
                self.words.extend(w)
                # add to documents in our corpus
                self.documents.append((pattern, intent.name))
                # add to our classes list
                if intent.name not in self.classes:
                    self.classes.append(intent.name)
        # stem and lower each word and remove duplicates
        self.words = [stemmer.stem(w.lower()) for w in self.words if w not in self.ignore_words]
        self.words = sorted(list(set(self.words)))

        # remove duplicates
        self.classes = sorted(list(set(self.classes)))

        print (len(self.documents), "documents")
        print (len(self.classes), "classes", self.classes)
        print (len(self.words), "unique stemmed words", self.words)
    def buildTrainingData(self):
        # create our training data
        training = []
        output = []
        # create an empty array for our output
        output_empty = [0] * len(self.classes)

        # training set, bag of words for each sentence
        x=[]
        y=[]
        for doc in self.documents:
            # initialize our bag of words
            bag = self.bow(doc[0])
            x.append(bag)
            # output is a '0' for each tag and '1' for current tag
            output_row = list(output_empty)
            output_row[self.classes.index(doc[1])] = 1
            y.append(output_row)
        # shuffle our features and turn into np.array
        random.shuffle(training)
        training = np.array(training)
        print(training.shape)

        # create train data
        self.train_x = np.array(x)
        self.train_y = np.array(y)
    def train(self):
        self.ann.train(self.train_x,self.train_y)
    def save(self):
        # save all of our data structures
        data={}
        data['words']    =self.words 
        data['classes']  =self.classes 
        data['documents']=self.documents 
        data['train_x']  =self.train_x 
        data['train_y']  =self.train_y
        pickle.dump( data, open( os.path.join('./',self.chatBot.name+".pk"), "wb" ) )
        self.ann.save(os.path.join('./',self.chatBot.name+'.h5'))    # guarda el modelo
    def load(self):
        data = pickle.load( open( os.path.join('./',self.chatBot.name+".pk"), "rb" ) )
        self.words     = data['words']
        self.classes   = data['classes']
        self.documents = data['documents']
        self.train_x   = np.array(data['train_x'])
        self.train_y   = np.array(data['train_y'])
        self.ann.load(os.path.join('./',self.chatBot.name+'.h5'))
    def predictClass(self,sentence):
        sentenceBow=self.bow(sentence)
        p=self.ann.predict(sentenceBow)
        idClass=np.argmax(p)
        pClass=np.max(p)
        className=self.classes[idClass]
        return className,idClass,pClass

In [102]:
class Intent(object):
    def __init__(self):
        self.name=""
        self.patterns=[]
        self.responses=[]
        self.action=None
    def fromJsonData(self,intent):
        self.patterns =[]
        self.responses=[]
        self.name=intent['tag']
        for pattern in intent['patterns']:
            self.patterns.append(pattern)
        for response in intent['responses']:
            self.responses.append(response)
        #self.action=intent['action']

In [124]:
class ChatBot(object):
    def __init__(self,name):
        self.name=name
        self.intents=[]
        self.model=NLPModelBoW()
        self.model.setChatBot(self)
        #TODO: refactor this
        self.loadJson()
        self.model.load()
    def loadJson(self):
        self.intents=[]
        with open(os.path.join('./',self.name+'.json')) as json_data:
            intents = json.load(json_data)
        # loop through each sentence in our intents patterns
        for intent in intents['intents']:
            iobj=Intent()
            iobj.fromJsonData(intent)
            self.intents.append(iobj)
        self.model.buildData()
    def chooseRandom(self,responses):
        sizeResponses=len(responses)
        chooseIdResponse=random.randint(0,sizeResponses-1)
        return responses[chooseIdResponse]
    def chooseResponse(self,predictedIntent):
        for intent in self.intents:
            if intent.name==predictedIntent:
                return self.chooseRandom(intent.responses)
    def chat(self,sentence):
        cn,idc,pc=self.model.predictClass(sentence)
        print(idc,cn,pc)
        if pc<0.55:
            return self.chooseRandom(["I don't understand your sentence.",
                                 "What do you mean?",
                                 "Could you please repeat with other words?"]),"do not understand"
        return self.chooseResponse(cn),cn

In [ ]:
class ChatBotEngine(object):
    def __init__(self):
        self.currentChatBot=ChatBot('intents')
        self.currentIntent="None"
    def getInput(self):
        return input('Ready: ')
    def setOutput(self,response):
        print(response)
    def run(self):
        while self.currentIntent!="goodbye":
            sentence=self.getInput()
            response,intent=self.currentChatBot.chat(sentence)
            self.currentIntent=intent
            self.setOutput(response)

In [ ]:
# These are analogous to winget for GUI
class Chatget(object):
    pass
# This is analogous to desktop GUI
class ChatDesktop(ChatBot):
    pass
# A chatbot to build or modify other chatbots
class MetaChatBot(ChatBot):
    pass
# Detect prediction error an improve the model of chatbots
class ChatLearner(ChatBot):
    pass

In [104]:
cb=ChatBot('intents')

27 documents
9 classes ['goodbye', 'greeting', 'hours', 'mopeds', 'opentoday', 'payments', 'rental', 'thanks', 'today']
48 unique stemmed words ["'d", "'s", 'a', 'acceiv', 'anyon', 'ar', 'bye', 'can', 'card', 'cash', 'credit', 'day', 'do', 'doe', 'good', 'goodby', 'hav', 'hello', 'help', 'hi', 'hour', 'how', 'i', 'is', 'kind', 'lat', 'lik', 'mastercard', 'mop', 'of', 'on', 'op', 'rent', 'see', 'tak', 'thank', 'that', 'ther', 'thi', 'to', 'today', 'we', 'what', 'when', 'which', 'work', 'yo', 'you']
(0,)


In [105]:
cb.intents[0].patterns

['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day']

In [106]:
test=cb.model.bow('Which mopeds do you have for sales?')
print(test.shape)
print(test)

(48,)
[0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 0 0 1]


In [107]:
cb.model.train_x.shape

(27, 48)

In [108]:
cb.model.train_y.shape

(27, 9)

In [82]:
cb.chat('Are you open now?')

2 hours 0.98776114


"We're open every day 9am-9pm"

In [109]:
cb.model.train()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 25)                1225      
_________________________________________________________________
dropout_4 (Dropout)          (None, 25)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 25)                650       
_________________________________________________________________
dropout_5 (Dropout)          (None, 25)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 9)                 234       
Total params: 2,109
Trainable params: 2,109
Non-trainable params: 0
_________________________________________________________________
Train on 27 samples
Epoch 1/500
27/27 [==============================] - 0s 10ms/sample - loss: 2.3469 - accuracy: 0.2

In [16]:
cb.model.save()

In [110]:
cb.model.load()

In [111]:
test=np.reshape(test,(1,48))
test.shape

(1, 48)

In [112]:
p=cb.model.ann.predict(test)

In [113]:
print(p)

[[3.14796011e-09 1.16214025e-07 3.47157822e-07 9.99977350e-01
  1.16851879e-06 6.39009886e-06 1.04172295e-05 3.39423173e-06
  8.38683832e-07]]


In [114]:
idclass=np.argmax(p)
print(idclass)

3


In [115]:
print(cb.model.classes[idclass])

mopeds


In [1]:
import pickle
import os
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.utils  import to_categorical

In [2]:
model = Sequential()
model.add(Dense(25, input_dim=27))          # densidad de la primera capa de neurona y tipo de entrada
model.add(Dropout(0.5))                                        # convierte a 0 la mitad de 1 en el entrenamiento
model.add(Dense(25))                                           # densidad de la primera capa de neurona
model.add(Dropout(0.5))                                        # convierte a 0 la mitad de 1 en el entrenamiento
model.add(Dense(9, activation='softmax'))  # densidad de la salida
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 25)                700       
_________________________________________________________________
dropout (Dropout)            (None, 25)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 25)                650       
_________________________________________________________________
dropout_1 (Dropout)          (None, 25)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 9)                 234       
Total params: 1,584
Trainable params: 1,584
Non-trainable params: 0
_________________________________________________________________
